# Calculate the biplot coordinates

If you have made it this far, give yourself a pat on the back -- that is a tricky step to retrieve the model parameters.

Next we will compute biplot coordinates so that we can visualize shiny biplots in Emperor.  This procedure will be available in qiime2 in the future -- but for now, we will have to do this calculation in a notebook.

There are only 2 parameters to tune below.  The first one is `dim`, which is the number of dimensions to approximate the conditional probability matrix (i.e. the ranks).  The second one is the scale aspect between the metabolite points and the microbe arrows.  To avoid things becoming too funky, the scale can either shrink the arrows (with scale < 1) or expand the arrows (with scale > 1).  This is mainly for aesthetic purposes.

In [1]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from biom import load_table
from skbio.stats.composition import clr, clr_inv, centralize, closure
from skbio.stats.ordination import OrdinationResults
from scipy.spatial.distance import euclidean

dim = 3      # --latent-dim
scale = 1.   # aspect ratios between metabolite points and microbe arrows

# read in the ranks.csv file we saved before.
ranks = pd.read_csv('../results/ranks.csv', index_col=0)

# Everything below are technical details.  You shouldn't have to touch this.
ranks = ranks.apply(clr_inv).T
u, s, v = np.linalg.svd(clr(centralize(ranks)))
n = v.shape[1]
u = u[:, :dim]
s = s[:dim]
v = v[:dim, :]

u = u * np.sqrt(n-1) / scale 
v = scale * np.diag(s) @ v / np.sqrt(n-1)

# push results into Ordination object
pcids = ['PC%d' % i for i in range(len(s))]
samples = pd.DataFrame(u[:, :len(s)], index=ranks.index, columns=pcids)
features = pd.DataFrame(v.T, index=ranks.columns, columns=pcids)
eigvals = pd.Series(s, index=pcids)

features['importance'] = features.apply(lambda x: euclidean(np.zeros_like(x), x), axis=1)
features.sort_values('importance', inplace=True, ascending=False)
features.drop(['importance'], inplace=True, axis=1)

res = OrdinationResults('MultiomicsBiplot', 'Multiomics Biplot',
                        eigvals = eigvals,
                        proportion_explained = eigvals / eigvals.sum(),
                        samples=samples, features=features)

# save that shit!
res.write('../results/cf_output/omics-biplot.results')

'../results/cf_output/omics-biplot.results'

If you want to load this into an emperor biplot, you can run the following commands in a qiime2 environment

```bash
qiime tools import --input-path ../results/omics-biplot.results \
                   --output-path ../results/omics-biplot.qza \
                   --type "PCoAResults % Properties('biplot')"
```

```bash
qiime emperor biplot --i-biplot ../results/omics-biplot.qza \
                     --m-sample-metadata-file ../data/lcms_annotations.txt \
                     --m-feature-metadata-file ../data/taxonomy.tsv \
                     --p-number-of-features 10 \
                     --o-visualization ../results/emperor-omics.qza
```

Are you getting an error from this command? Turns out that you need to make sure that `sampleid` is the name of the first column in `lcms_annotations.txt` and `featureid` is the name of the first column in `taxonomy.tsv`.  That can be done as follows.

In [2]:
import os
import numpy as np
import pandas as pd

data_dir = '../data/'

taxonomy = pd.read_table(os.path.join(data_dir, 'taxonomy.tsv'), index_col=0)
ms_md = pd.read_table(os.path.join(data_dir, 'lcms_annotations.txt'))

ms_md.index.name = 'sampleid'
ms_md.to_csv('../results/point-metadata.txt', sep='\t')

taxonomy.index.name = 'featureid'
taxonomy.to_csv('../results/arrow-metadata.txt', sep='\t')

Are you still getting errors after this? To get around that, you can run --p-ignore-missing-samples to drop any samples that aren't shared between the metadata mapping files.

We will go more in-depth on how to create beautiful metadata to paint your Emperor biplots. But first, we will cover how to customize biplots so that you can place custom labels directly on your plot. Note that this won't be 3D or interactive -- but it allows for more customizability at the moment (but be sure to watch out for updates with Emperor, since it is under active development).